In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
tqdm.pandas()

In [22]:
# df_name = 'B5_jan-dec2021.csv'
# df_name = 'B5_jan-mar2022.csv'
# df_name = 'corona_news_sep-mar2022.csv'
df_name1 = 'B5_sep-dec2020.csv'
df_name2 = 'NS_sep-dec2020.csv'

In [23]:
predict_df1 = pd.read_csv('csv/analysis_' + df_name1)
predict_df2 = pd.read_csv('csv/analysis_' + df_name2)
predict_df = pd.concat((predict_df1, predict_df2),axis=0)
predict_date = predict_df.date
predict_df.date = pd.to_datetime(predict_df.date)

In [24]:
def change(x):
    if x == 0:
        return -1
    else:
        return 1
predict_df['prediction'] = predict_df.prediction.apply(lambda x: change(x))
predict_df['weighted_prediction'] = predict_df['prediction'] * predict_df['score']

In [25]:
predict_df = predict_df[predict_df.score > 0.80]

In [26]:
predict_df = predict_df.groupby('article_id').mean()[['prediction', 'weighted_prediction']]
predict_df.reset_index(level=0, inplace=True)
print(f'Number of unique articles being analyzed: {predict_df.article_id.nunique()}')

Number of unique articles being analyzed: 28800


In [27]:
def pre_explode_merge(df_name):
    pre_explode1 = pd.read_csv('csv/pre_explode_' + df_name2)
    pre_explode2 = pd.read_csv('csv/pre_explode_' + 'B5_sep2020-mar2022.csv') # Fill in for one time. 
    pre_explode = pd.concat((pre_explode1,pre_explode2),axis=0)
    
    pre_explode.dropna(inplace=True)
    pre_explode.date = pd.to_datetime(pre_explode.date)    
    pre_explode = pre_explode[(pre_explode.date >= predict_date.min()) & (pre_explode.date <= predict_date.max())]
    for_comparison = predict_df.article_id.unique()
    pre_explode = pre_explode[pre_explode.article_id.isin(for_comparison)] # Makes sure pre-explode has all the same articles.
    pre_explode.drop(pre_explode.columns[:1],axis=1,inplace=True)
    return pre_explode

In [28]:
pre_explode = pre_explode_merge(df_name1)

In [29]:
# Make sure this is equal to zero
pre_explode.article_id.nunique() - predict_df.article_id.nunique()

0

In [30]:
df = predict_df.merge(pre_explode, on='article_id')

In [31]:
df = df.groupby('date').mean().reset_index(level=0)
df

,date,prediction,weighted_prediction
0,2020-09-01,-0.198547,-0.201171
1,2020-09-02,-0.212272,-0.213734
2,2020-09-03,-0.187986,-0.189249
3,2020-09-04,-0.250412,-0.249771
4,2020-09-05,-0.277693,-0.276330
...,...,...,...
117,2020-12-27,-0.222319,-0.225316
118,2020-12-28,-0.245833,-0.248244
119,2020-12-29,-0.263309,-0.263561
120,2020-12-30,-0.151042,-0.153904


In [32]:
save_file = 'email_analysis/' + df_name2
df.to_csv(save_file)

In [33]:
save_file

'email_analysis/NS_sep-dec2020.csv'